# Handwritten Digit Recognition with Machine Learning

Just attempting to try handwritten digit recognition with machine learning.

Project inspired by scikit-learn's "Recognizing hand-written digits": https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py

## Import libraries

In [ ]:
import numpy as np
from numpy import asarray
import os
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import tkinter as tk
from PIL import Image
from PIL import ImageGrab

## Import the images and labels. 

In [ ]:
train_images = np.load('Data/trainImages.npy')
train_labels = np.load('Data/trainLabels.npy')

## Training the models.

In [ ]:
#----------Random Forest----------
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf_model.fit(train_images, train_labels)

In [ ]:
#----------K-Nearest Neighbours----------
knn_model = KNeighborsClassifier(n_neighbors = 10, weights = "distance")
knn_model.fit(train_images, train_labels)

In [ ]:
#----------Support Vector Machine (SVM)----------
svm_model = SVC(kernel = "rbf", C = 5, random_state = 1)
svm_model.fit(train_images, train_labels)

## GUI For Canvas 

<b>NOTE:</b> BEFORE RUNNING THE CODE, go on Display Settings -> Scale and layout -> then "Change the size of text, apps and other items" to 100% (for Windows 10). Otherwise, fetching the co-ordinates of the canvas will be inaccurate.

## Methods to help run the GUI

In [ ]:
# Declare certain variables. 
canvasWidth = 280
canvasHeight = 280
borderThickness = 2
mousePos = { "x":0, "y":0 }
lineWidth = 15
font_size = 12
save_directory = "Data/Digits/"

# Sets the current position of the mouse.
def setPosition(event):
    mousePos["x"] = event.x 
    mousePos["y"] = event.y

# Draw on the canvas based on the current position of the mouse.
def draw(event):
    currX = mousePos["x"]
    currY = mousePos["y"]
    canvas.create_oval(currX, currY, currX, currY, width = lineWidth)
    setPosition(event)

# Clears the canvas and hides any applicable elements.
def clearCanvas():
    canvas.delete("all")
    btnWrongPred.pack_forget()
    lblRandForest.config(text = "")
    lblKNN.config(text = "")
    lblSVM.config(text = "")
    
# Takes a screenshot of the canvas and returns it.
def getCanvasScreenshot():
    canvas.update()
    
    # Added (borderThickness) to prevent the border from being part of the screenshot.
    x0 = canvas.winfo_rootx() + borderThickness
    y0 = canvas.winfo_rooty() + borderThickness
    x1 = x0 + canvasWidth
    y1 = y0 + canvasHeight
    
    screenShot = ImageGrab.grab((x0, y0, x1, y1))
    return screenShot

# Displays the window at the centre of the screen.
def centreWindow(width, height, window):
    window_width = width
    window_height = height
    x_window_pos = (window.winfo_screenwidth() / 2) - (window_width / 2)
    y_window_pos = (window.winfo_screenheight() / 2) - (window_height / 2)
    window.geometry("%dx%d+%d+%d" % (window_width, window_height, x_window_pos, y_window_pos))
    
# Makes the predictions on the image.
def predictImage():
    # Get screenshot and adjust the image.
    screenShot = getCanvasScreenshot()
    screenShot = screenShot.convert('P', palette = Image.ADAPTIVE, colors = 2)
    
    # Convert the image into a numpy array.
    npImg = asarray(screenShot, dtype = "int").reshape(1, -1)
    
    # Remove '[0]' if you don't mind the brackets (in the output).
    rf_pred = rf_model.predict(npImg)[0]
    knn_pred = knn_model.predict(npImg)[0]
    svm_pred = svm_model.predict(npImg)[0]
    
    # Display the predictions on the GUI.
    lblRandForest.config(text = "Random Forest: " + str(rf_pred))
    lblKNN.config(text = "K-Nearest Neighbours: " + str(knn_pred))
    lblSVM.config(text = "SVM: " + str(svm_pred))
    btnWrongPred.pack() # Show the button.

# Displays a seperate window that allows the user to submit what was drawn
# if the user believes that the prediction(s) was wrong.
def openPredictWindow():
    predictWindow = tk.Toplevel(root)
    window_width = 200
    window_height = 120
    centreWindow(window_width, window_height, predictWindow)
    predictWindow.resizable(0, 0)
    predictWindow.grab_set() # Prevent interaction of the main window.
    
    lblPredictPrompt = tk.Label(predictWindow, text = "Enter the number you drew:", 
                                font = ("Calibri", font_size)).pack()
    
    txtPredict = tk.Entry(predictWindow, font = ("Calibri", font_size))
    txtPredict.pack()
    
    lblInvalidInput = tk.Label(predictWindow, font = ("Calibri", font_size))
    lblInvalidInput.pack()
    
    # Lambda allows functions to pass down parameters in 'command'.
    btnSubmit = tk.Button(predictWindow, text = "Submit", font = ("Calibri", font_size), 
                          command = lambda : saveImage(txtPredict.get(), predictWindow, lblInvalidInput)).pack()

# Saves the canvas contents as a new training image, then closes the sub-window.
def saveImage(userPred, predictWindow, lblInvalidInput):
    # Remove whitespace (it's a single digit input)
    userPred = userPred.replace(" ", "") 
    
    # The input must be a single digit.
    if (not userPred.isnumeric()) or (len(userPred) > 1):
        lblInvalidInput.config(foreground = "red", text = "Invalid Input")
    else:
        # Close the window and wait (for it to close).
        # This avoids the window from being in the screenshot (if it is).
        predictWindow.destroy()
        time.sleep(1)
        
        # Save the image (name is based on number of files in the respective folder).
        path, dirs, files = next(os.walk(save_directory + userPred))
        file_count = len(files)
        screenShot = getCanvasScreenshot()
        screenShot.save(save_directory + userPred + "/" + str(file_count + 1) + ").png")

## Run the GUI 

Declaring all the tkinter objects for the main window.

In [ ]:
# Create the main window and add its elements.
root = tk.Tk()
window_width = 500
window_height = 500
centreWindow(window_width, window_height, root)
root.resizable(0, 0)

canvas = tk.Canvas(root, bg = "white", width = canvasWidth, height = canvasHeight, 
                   highlightthickness = borderThickness, highlightbackground = "black")
canvas.bind("<Button-1>", setPosition)
canvas.bind("<B1-Motion>", draw)
canvas.pack()

# Set frames for the buttons to help position them.
topButtonFrame = tk.Frame(root)
bottomButtonFrame = tk.Frame(root)
topButtonFrame.pack()
bottomButtonFrame.pack()

# Create and display each button (apart from 'btnWrongPred').
btnClear = tk.Button(topButtonFrame, text = "Clear", font = ("Calibri", font_size), 
                     command = clearCanvas)
btnClear.pack(side = "left")
btnPredict= tk.Button(topButtonFrame, text = "Predict", font = ("Calibri", font_size), 
                      command = predictImage)
btnPredict.pack(side = "right")
btnWrongPred = tk.Button(bottomButtonFrame, text = "Wrong Prediction?", font = ("Calibri", font_size), 
                         command = openPredictWindow)

# Create and display the labels (with no content).
lblRandForest = tk.Label(root, font = ("Calibri", font_size))
lblKNN = tk.Label(root, font = ("Calibri", font_size))
lblSVM = tk.Label(root, font = ("Calibri", font_size))
lblRandForest.pack()
lblKNN.pack()
lblSVM.pack()

root.mainloop() # Run it.

## NOTE: The following code beyond this point is the pre-processing of the training data and labels.

## Convert images into a numpy array
Folders are ordered from 0 to 9 by default, so the labels can be determined based on that. <b> RUN THIS EVERYTIME NEW TRAINING IMAGES ARE ADDED. </b>

In [ ]:
digits_directory = "Data/Digits/"
imgList = []
labels = []
i = 0

# Search through each image folder.
for path, dirs, files in os.walk(digits_directory):
    # Go through each image (from each folder) and add them to the list.
    for file in files:
        if file.endswith('.png'):
            # Reduce to a single colour depth as well (it's a black and white image).
            img = Image.open(digits_directory + str(i) + "/" + file).convert('P', palette = Image.ADAPTIVE, colors = 2)
            img = asarray(img).reshape(-1, 1)
            imgList.append(img)
            labels.append(i)
    # Only increment if the folder isn't empty.
    # 'files' can hold empty folders for some reason...
    if len(files) > 0:
        i = i + 1

# Convert images.
n_samples = len(imgList)
npImages = asarray(imgList, dtype = "int").reshape((n_samples, -1))
np.save("Data/trainImages", npImages)

# Convert labels.
npLabels = asarray(labels, dtype = "int")
np.save("Data/trainLabels", npLabels)   
#print(npImages.shape)

## Testing image imports

Bits of code to experiment what the images look like.

In [ ]:
# Converts it to a single colour depth.
img = Image.open('two.png').convert('P', palette = Image.ADAPTIVE, colors = 50)
img.thumbnail((28, 28), Image.ANTIALIAS)
img.show()
npImg = asarray(img).reshape(1, -1)

pred = rf_model.predict(npImg)
print(pred)

In [ ]:
print(npImg.shape)
plt.imshow(img)
plt.show()